# Criação da tabela `match_features`

Este notebook cria a tabela `match_features` a partir:

- da tabela `Match` (placar, times, liga, temporada), e
- das tabelas de eventos normalizadas:

  - `shoton_events`, `shotoff_events`
  - `corner_events`
  - `cross_events`
  - `foulcommit_events`
  - `card_events`
  - `possession_events`

A tabela `match_features` contém, para cada partida:

- informações de contexto:
  - `match_id`, `league_id`, `season`
  - `home_team_api_id`, `away_team_api_id`
  - `home_team_goal`, `away_team_goal`
  - `goal_diff` (saldo de gols do mandante)
  - `result_label` (1 = vitória do mandante, 0 = empate, -1 = vitória do visitante)
- estatísticas agregadas para mandante e visitante:
  - chutes, chutes no alvo
  - escanteios, cruzamentos
  - faltas
  - cartões amarelos e vermelhos
  - índice de cartões (`cards_index_*`)
  - posse de bola (`possession_home`, `possession_away`, `possession_diff`)

Essa tabela será usada nos notebooks seguintes para gerar:
- a visão por time (`team_match_features`), e
- as janelas temporais de desempenho.


In [ ]:
# 1. Imports e conexão com o banco SQLite

import sqlite3

# Caminho do banco (ajuste se necessário)
DB_PATH = "../data/database.sqlite"

conn = sqlite3.connect(DB_PATH)
print("Conexão aberta com o banco.")


In [ ]:
# 2. SQL para criação da tabela match_features

sql_match_features = """
CREATE TABLE IF NOT EXISTS match_features AS
WITH selected_matches AS (
  SELECT
    m.id AS match_id,
    m.league_id,
    m.season,
    m.home_team_api_id,
    m.away_team_api_id,
    m.home_team_goal,
    m.away_team_goal
  FROM Match m
  WHERE m.league_id IN (1729, 4769, 7809, 10257, 13274, 21518, 24558)
    AND m.season BETWEEN '2008/2009' AND '2015/2016'
),
shots AS (
  SELECT
    m.match_id,
    SUM(CASE WHEN CAST(e.team AS INTEGER) = m.home_team_api_id THEN 1 ELSE 0 END) AS shots_home,
    SUM(CASE WHEN CAST(e.team AS INTEGER) = m.away_team_api_id THEN 1 ELSE 0 END) AS shots_away
  FROM selected_matches m
  LEFT JOIN (
    SELECT match_id, team FROM shoton_events
    UNION ALL
    SELECT match_id, team FROM shotoff_events
  ) e
    ON e.match_id = m.match_id
  GROUP BY m.match_id
),
shots_on AS (
  SELECT
    m.match_id,
    SUM(CASE WHEN CAST(s.team AS INTEGER) = m.home_team_api_id THEN 1 ELSE 0 END) AS shots_on_home,
    SUM(CASE WHEN CAST(s.team AS INTEGER) = m.away_team_api_id THEN 1 ELSE 0 END) AS shots_on_away
  FROM selected_matches m
  LEFT JOIN shoton_events s
    ON s.match_id = m.match_id
  GROUP BY m.match_id
),
corners AS (
  SELECT
    m.match_id,
    SUM(CASE WHEN CAST(c.team AS INTEGER) = m.home_team_api_id THEN 1 ELSE 0 END) AS corners_home,
    SUM(CASE WHEN CAST(c.team AS INTEGER) = m.away_team_api_id THEN 1 ELSE 0 END) AS corners_away
  FROM selected_matches m
  LEFT JOIN corner_events c
    ON c.match_id = m.match_id
  GROUP BY m.match_id
),
crosses AS (
  SELECT
    m.match_id,
    SUM(CASE WHEN CAST(c.team AS INTEGER) = m.home_team_api_id THEN 1 ELSE 0 END) AS crosses_home,
    SUM(CASE WHEN CAST(c.team AS INTEGER) = m.away_team_api_id THEN 1 ELSE 0 END) AS crosses_away
  FROM selected_matches m
  LEFT JOIN cross_events c
    ON c.match_id = m.match_id
  GROUP BY m.match_id
),
fouls AS (
  SELECT
    m.match_id,
    SUM(CASE WHEN CAST(f.team AS INTEGER) = m.home_team_api_id THEN 1 ELSE 0 END) AS fouls_home,
    SUM(CASE WHEN CAST(f.team AS INTEGER) = m.away_team_api_id THEN 1 ELSE 0 END) AS fouls_away
  FROM selected_matches m
  LEFT JOIN foulcommit_events f
    ON f.match_id = m.match_id
  GROUP BY m.match_id
),
cards AS (
  SELECT
    m.match_id,
    COALESCE(SUM(CASE WHEN CAST(c.team AS INTEGER) = m.home_team_api_id AND c.card_type IN ('y','y2') THEN 1 ELSE 0 END),0) AS yellows_home,
    COALESCE(SUM(CASE WHEN CAST(c.team AS INTEGER) = m.away_team_api_id AND c.card_type IN ('y','y2') THEN 1 ELSE 0 END),0) AS yellows_away,
    COALESCE(SUM(CASE WHEN CAST(c.team AS INTEGER) = m.home_team_api_id AND c.card_type = 'r' THEN 1 ELSE 0 END),0) AS reds_home,
    COALESCE(SUM(CASE WHEN CAST(c.team AS INTEGER) = m.away_team_api_id AND c.card_type = 'r' THEN 1 ELSE 0 END),0) AS reds_away
  FROM selected_matches m
  LEFT JOIN card_events c
    ON c.match_id = m.match_id
  GROUP BY m.match_id
),
poss AS (
  SELECT
    p.match_id,
    AVG(CAST(p.comment AS INTEGER)) AS possession_home
  FROM possession_events p
  GROUP BY p.match_id
)
SELECT
  sm.match_id,
  sm.league_id,
  sm.season,
  sm.home_team_api_id,
  sm.away_team_api_id,
  sm.home_team_goal,
  sm.away_team_goal,
  (sm.home_team_goal - sm.away_team_goal) AS goal_diff,
  CASE
    WHEN sm.home_team_goal > sm.away_team_goal THEN 1
    WHEN sm.home_team_goal = sm.away_team_goal THEN 0
    ELSE -1
  END AS result_label,
  shots.shots_home,
  shots.shots_away,
  (shots.shots_home - shots.shots_away) AS shots_diff,
  shots_on.shots_on_home,
  shots_on.shots_on_away,
  (shots_on.shots_on_home - shots_on.shots_on_away) AS shots_on_diff,
  corners.corners_home,
  corners.corners_away,
  (corners.corners_home - corners.corners_away) AS corners_diff,
  crosses.crosses_home,
  crosses.crosses_away,
  (crosses.crosses_home - crosses.crosses_away) AS crosses_diff,
  fouls.fouls_home,
  fouls.fouls_away,
  (fouls.fouls_home - fouls.fouls_away) AS fouls_diff,
  cards.yellows_home,
  cards.yellows_away,
  cards.reds_home,
  cards.reds_away,
  (cards.yellows_home + 2*cards.reds_home) AS cards_index_home,
  (cards.yellows_away + 2*cards.reds_away) AS cards_index_away,
  ((cards.yellows_home + 2*cards.reds_home) - (cards.yellows_away + 2*cards.reds_away)) AS cards_index_diff,
  poss.possession_home,
  CASE WHEN poss.possession_home IS NOT NULL
       THEN 100.0 - poss.possession_home
       ELSE NULL
  END AS possession_away,
  CASE WHEN poss.possession_home IS NOT NULL
       THEN poss.possession_home - (100.0 - poss.possession_home)
       ELSE NULL
  END AS possession_diff
FROM selected_matches sm
LEFT JOIN shots     ON shots.match_id     = sm.match_id
LEFT JOIN shots_on  ON shots_on.match_id  = sm.match_id
LEFT JOIN corners   ON corners.match_id   = sm.match_id
LEFT JOIN crosses   ON crosses.match_id   = sm.match_id
LEFT JOIN fouls     ON fouls.match_id     = sm.match_id
LEFT JOIN cards     ON cards.match_id     = sm.match_id
LEFT JOIN poss      ON poss.match_id      = sm.match_id;
"""


In [ ]:
# 3. Executar o SQL para criar/atualizar a tabela match_features

# (Opcional) Derrubar a tabela antiga para recriar do zero:
conn.execute("DROP TABLE IF EXISTS match_features;")
conn.commit()

conn.executescript(sql_match_features)
conn.commit()

print("Tabela 'match_features' criada com sucesso.")


In [ ]:
# 4. Verificar a tabela match_features

cur = conn.cursor()

# Quantidade de linhas
n = cur.execute("SELECT COUNT(*) FROM match_features;").fetchone()[0]
print("Total de linhas em match_features:", n)

# Primeiras linhas (só para conferir estrutura)
for row in cur.execute("SELECT * FROM match_features LIMIT 5;"):
    print(row)

conn.close()
print("Conexão fechada.")
